##### Copyright 2018 The TensorFlow Hub Authors.

##### Modifications Copyright 2019 Tomoaki Masuda.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

このノートブックは、以下のノートブックを元に日本語訳、一部章立ての再構成、加筆を行いました。https://github.com/tensorflow/hub/blob/master/examples/colab/action_recognition_with_tf_hub.ipynb


#  TF-Hub 動画のアクション認識モデル



このノートブックでは、[tfhub.dev/deepmind / i3d-kinetics-400/1](https://tfhub.dev/deepmind/i3d-kinetics-400/1)モジュールを使って、動画データからのアクション認識を試します。

基礎となるモデルは、Joao
Carreira氏とAndrew Zisserman氏による "[Quo Vadis, Action Recognition? A New
Model and the Kinetics Dataset](https://arxiv.org/abs/1705.07750)" で説明されています。
この論文は2017年5月にarXivに投稿され、CVPR 2017論文として発表されました。 ソースコードは [github](https://github.com/deepmind/kinetics-i3d) で公開されています。 

 "Quo Vadis"では、動画分類のための新しいアーキテクチャ、Inflated 3D Convnet、略してI3Dを導入しました。このアーキテクチャをファインチューニング（転移学習）したもので、UCF101データセット、およびHMDB51データセットで最高精度を達成しました。

Kineticsで事前学習したI3Dモデルは、CVPR 2017 [Charades challenge](http://vuchallenge.org/charades.html)でも優勝しました。

元のモジュールは[kinetics-400データセット](https://deepmind.com/research/open-source/open-source-datasets/kinetics/) で学習し、約400の異なる動作を識別できます。これらのアクションのラベルは、[label map file](https://github.com/deepmind/kinetics-i3d/blob/master/data/label_map.txt) で確認できます。

このノートブックでは、UCF101データセットのビデオ内の動作を認識します。


## A. 環境を準備する

必要なライブラリのインストール、インポートを行います。

In [ ]:
#added on 2020/6/20
%tensorflow_version 1.x

# Install the necessary python packages.
!pip install -q "tensorflow>=1.7" "tensorflow-hub" "imageio"

In [ ]:
#@title Import the necessary modules
# TensorFlow and TF-Hub modules.
import tensorflow as tf
import tensorflow_hub as hub

tf.logging.set_verbosity(tf.logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import cv2
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3


## B. データセットを準備する


### 1. UCF101データセットをダウンロードする

UCF101データセットをダウンロードします。オリジナルのノートブックはリンク切れとなっていたため、変更を行なっています。

In [ ]:
#@title Helper functions for the UCF101 dataset

# Utilities to fetch videos from UCF101 dataset
UCF_ROOT = "http://crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()

def list_ucf_videos():
  """Lists videos available in UCF101 dataset."""
  global _VIDEO_LIST
  if not _VIDEO_LIST:
    index = request.urlopen(UCF_ROOT).read().decode("utf-8")
    videos = re.findall("(v_[\w_]+\.avi)", index)
    _VIDEO_LIST = sorted(set(videos))
  return list(_VIDEO_LIST)

def fetch_ucf_video(video):
  """Fetchs a video and cache into local filesystem."""
  cache_path = os.path.join(_CACHE_DIR, video)
  if not os.path.exists(cache_path):
    urlpath = request.urljoin(UCF_ROOT, video)
    print("Fetching %s => %s" % (urlpath, cache_path))
    data = request.urlopen(urlpath).read()
    open(cache_path, "wb").write(data)
  return cache_path

# Utilities to open video files using CV2
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      
      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0

def animate(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=25)
  with open('./animation.gif','rb') as f:
      display.display(display.Image(data=f.read(), height=300))


### 2. kinetics-400のラベルを取得する


In [ ]:
#@title Get the kinetics-400 labels
# Get the kinetics-400 action labels from the GitHub repository.
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
with request.urlopen(KINETICS_URL) as obj:
  labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found %d labels." % len(labels))


### 3. データセットの中身を見てみる

まず、UCF101データセットに含まれる動画のリストを取得します。さらに、サンプルの動画を再生してみましょう。


In [ ]:
# Get the list of videos in the dataset.
ucf_videos = list_ucf_videos()
  
categories = {}
for video in ucf_videos:
  category = video[2:-12]
  if category not in categories:
    categories[category] = []
  categories[category].append(video)
print("Found %d videos in %d categories." % (len(ucf_videos), len(categories)))

for category, sequences in categories.items():
  summary = ", ".join(sequences[:2])
  print("%-20s %4d videos (%s, ...)" % (category, len(sequences), summary))


In [ ]:
# Get a sample cricket video.
sample_video = load_video(fetch_ucf_video("v_CricketShot_g04_c02.avi"))

print("sample_video is a numpy array of shape %s." % str(sample_video.shape))
animate(sample_video)

## C. データセットを前処理する（不要）



## D. モデルを取得する

TF-Hubから、学習済みのI3Dモデルを取得します。


## E. モデルを学習させる

学習済みモデルを使うため、ここでは行いません。

## F. 学習済みモデルを評価する

先ほど取得した学習済みモデルを使って、指定した動画の動作カテゴリを認識してみます。

In [ ]:
# Run the i3d model on the video and print the top 5 actions.

# First add an empty dimension to the sample video as the model takes as input
# a batch of videos.
model_input = np.expand_dims(sample_video, axis=0)

# Create the i3d model and get the action probabilities.
with tf.Graph().as_default():
  i3d = hub.Module("https://tfhub.dev/deepmind/i3d-kinetics-400/1")
  input_placeholder = tf.placeholder(shape=(None, None, 224, 224, 3), dtype=tf.float32)
  logits = i3d(input_placeholder)
  probabilities = tf.nn.softmax(logits)
  with tf.train.MonitoredSession() as session:
    [ps] = session.run(probabilities,
                       feed_dict={input_placeholder: model_input})

print("Top 5 actions:")
for i in np.argsort(ps)[::-1][:5]:
  print("%-22s %.2f%%" % (labels[i], ps[i] * 100))